<a href="https://colab.research.google.com/github/prathaban-sr/eva6/blob/main/EVA_Session_4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [140]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [141]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 24, 3) #26 -? OUtput? 3
        self.bn1 = nn.BatchNorm2d(24)    
        self.conv2 = nn.Conv2d(24, 32, 3) #24,,5
        self.pool1 = nn.MaxPool2d(2, 2)#12,,10
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 42, 3)#10,,12
        self.bn3 = nn.BatchNorm2d(42)
        self.pool2 = nn.AvgPool2d(10, 10)#1,,28
        self.fc1 = nn.Linear(42,10)
        self.dropout = nn.Dropout(0.05)

    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.dropout(x)
        
        x = self.bn2(F.relu(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout(x)

        x = F.relu(self.conv3(x))
        x = self.bn3(x)
        x = self.pool2(x)
        x = self.dropout(x)
        x = x.view(-1,42)

        x = self.fc1(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [142]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 26, 26]             240
       BatchNorm2d-2           [-1, 24, 26, 26]              48
           Dropout-3           [-1, 24, 26, 26]               0
            Conv2d-4           [-1, 32, 24, 24]           6,944
       BatchNorm2d-5           [-1, 32, 24, 24]              64
         MaxPool2d-6           [-1, 32, 12, 12]               0
           Dropout-7           [-1, 32, 12, 12]               0
            Conv2d-8           [-1, 42, 10, 10]          12,138
       BatchNorm2d-9           [-1, 42, 10, 10]              84
        AvgPool2d-10             [-1, 42, 1, 1]               0
          Dropout-11             [-1, 42, 1, 1]               0
           Linear-12                   [-1, 10]             430
Total params: 19,948
Trainable params: 19,948
Non-trainable params: 0
---------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [143]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [144]:
#from tqdm import tqdm
# def train(model, device, train_loader, optimizer):
#     model.train()
#     #pbar = tqdm(train_loader)
#     for epoch in range(19):
#       correct = 0
#       loss = 0
    
#       for data, target in train_loader:
#           data, target = data.to(device), target.to(device)
#           optimizer.zero_grad()
#           output = model(data)
#           loss = F.nll_loss(output, target)
#           loss.backward()
#           optimizer.step()
#           pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
#           correct += pred.eq(target.view_as(pred)).sum().item()

#         #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

#       #loss /= len(train_loader.dataset)

#       print('\nTrain set: Accuracy: {}/{} ({:.4f}%)'.format(
#         correct, len(train_loader.dataset),
#           100. * correct / len(train_loader.dataset)))

# def test(model, device, test_loader):
#     model.eval()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
#             pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
#             correct += pred.eq(target.view_as(pred)).sum().item()

#     test_loss /= len(test_loader.dataset)

#     print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
#         test_loss, correct, len(test_loader.dataset),
#         100. * correct / len(test_loader.dataset)))

In [145]:
#every epoch tracking
from tqdm import tqdm
def train(model, device, train_loader, optimizer):
    #pbar = tqdm(train_loader)
    for epoch in range(19):
      model.train()
      correct = 0
      loss = 0
    
      for data, target in train_loader:
          data, target = data.to(device), target.to(device)
          optimizer.zero_grad()
          output = model(data)
          loss = F.nll_loss(output, target)
          loss.backward()
          optimizer.step()
          pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
          correct += pred.eq(target.view_as(pred)).sum().item()

        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

      print('\nTrain set: Accuracy: {}/{} ({:.4f}%)'.format(
        correct, len(train_loader.dataset),
          100. * correct / len(train_loader.dataset)))

      model.eval()
      test_loss = 0
      test_correct = 0
      with torch.no_grad():
          for data, target in test_loader:
              data, target = data.to(device), target.to(device)
              output = model(data)
              test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
              pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
              test_correct += pred.eq(target.view_as(pred)).sum().item()

      test_loss /= len(test_loader.dataset)

      print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
          test_loss, test_correct, len(test_loader.dataset),
          100. * test_correct / len(test_loader.dataset)))
    
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [146]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.25, momentum=0.9)
#optimizer = optim.Adam(model.parameters(), lr=0.25)

#for epoch in range(1, 2):
train(model, device, train_loader, optimizer)
test(model, device, test_loader)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Train set: Accuracy: 55620/60000 (92.7000%)

Test set: Average loss: 0.0825, Accuracy: 9758/10000 (97.5800%)


Train set: Accuracy: 58610/60000 (97.6833%)

Test set: Average loss: 0.0655, Accuracy: 9810/10000 (98.1000%)


Train set: Accuracy: 58903/60000 (98.1717%)

Test set: Average loss: 0.0512, Accuracy: 9833/10000 (98.3300%)


Train set: Accuracy: 59029/60000 (98.3817%)

Test set: Average loss: 0.0457, Accuracy: 9850/10000 (98.5000%)


Train set: Accuracy: 59138/60000 (98.5633%)

Test set: Average loss: 0.0435, Accuracy: 9871/10000 (98.7100%)


Train set: Accuracy: 59236/60000 (98.7267%)

Test set: Average loss: 0.0422, Accuracy: 9872/10000 (98.7200%)


Train set: Accuracy: 59256/60000 (98.7600%)

Test set: Average loss: 0.0374, Accuracy: 9883/10000 (98.8300%)


Train set: Accuracy: 59347/60000 (98.9117%)

Test set: Average loss: 0.0363, Accuracy: 9875/10000 (98.7500%)


Train set: Accuracy: 59344/60000 (98.9067%)

Test set: Average loss: 0.0392, Accuracy: 9875/10000 (98.7500%)


